In [1]:
import pandas as pd 
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

C:\Users\gudiy\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gudiy\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gudiy\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\gudiy\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X

[[52, 78, 341, 456, 22, 2, 420, 365, 95, 29, 51, 1039, 1],
 [351,
  125,
  1954,
  2,
  597,
  1,
  29,
  51,
  228,
  35,
  198,
  5,
  175,
  1417,
  10,
  1577,
  1356,
  847],
 [61, 498, 5, 8, 2, 1, 21, 1797, 10, 696, 518],
 [17, 278, 238, 6, 736, 96, 160, 24, 132, 5, 2, 179, 10, 1, 214, 786, 16],
 [1239, 1, 291, 23, 2, 1685, 236, 10, 2, 669, 5, 176, 188, 403, 563],
 [118,
  16,
  47,
  211,
  333,
  9,
  70,
  274,
  421,
  1357,
  1955,
  1194,
  63,
  1956,
  192,
  1,
  56],
 [9,
  1240,
  160,
  8,
  21,
  1418,
  64,
  9,
  606,
  184,
  21,
  168,
  4,
  32,
  2,
  632,
  20,
  1008,
  1,
  49,
  822],
 [168, 16, 759, 404, 42, 135, 633, 182, 1578, 1],
 [10, 2, 167, 102, 535, 32, 2, 101, 607, 45, 57, 1798, 7, 16, 658, 1],
 [29, 116, 24, 881, 14, 1, 915],
 [23, 10, 48, 17],
 [1799, 38, 1957, 46, 4, 598, 19, 65, 1, 175, 608, 5, 2, 51, 161, 325],
 [34, 1148, 47, 457, 22, 52, 153, 2, 1, 21],
 [1149, 346, 293, 1, 1958],
 [619, 9, 609, 316, 174, 55, 1959, 10, 1500, 183, 14, 46, 1],

In [3]:
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   51, 1039,    1],
       [   0,    0,    0, ..., 1577, 1356,  847],
       [   0,    0,    0, ...,   10,  696,  518],
       ...,
       [   0,    0,    0, ...,   68,   62,    3],
       [   0,    0,    0, ..., 1112, 1588,   81],
       [   0,    0,    0, ...,  196,    3,  880]])

In [4]:
embed_dim = 128
lstm_out = 196
tbCallBack= keras.callbacks.TensorBoard(log_dir='./Graph', write_graph=True, write_images=True)
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [5]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [6]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


Epoch 1/1
 - 30s - loss: 0.8471 - accuracy: 0.6354
0.7772661416928495
0.6548711061477661
['loss', 'accuracy']


In [7]:
model1_json = model.to_json()
with open('model1.json', 'w') as json_file:
    json_file.write(model1_json)
model.save_weights('model1.h5')

In [8]:
from keras.models import model_from_json
# load YAML and create model
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
# predict data
twt = 'A lot of good things are happening. We are respected again throughout the world, and that\'s a great thing'
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(twt)

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)

#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)

In [14]:
import numpy as np
print(twt)
result = loaded_model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(result) == 0):
    print("negative")
elif (np.argmax(result) == 1):
    print("positive")


[[ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0 13]
 ...
 [ 0  0  0 ...  0  0 10]
 [ 0  0  0 ...  0  0  7]
 [ 0  0  0 ...  0  0  5]]
positive
